<a href="https://colab.research.google.com/github/nanopiero/exam_S3/blob/master/notebooks/Multimodal_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atelier 4 : Régression multimodale avec un Visual Transformer


In [5]:
# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

In [1]:
! git clone https://github.com/nanopiero/exam_S3.git

Cloning into 'exam_S3'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 19.86 KiB | 19.86 MiB/s, done.


In [2]:
pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 757.4 kB/s eta 0:00:00


## A. Découverte du problème

Dans ce problème, il va s'agir de reconstruire un champ 2D à partir de plusieurs sources d'information. Les sources d'information sont les suivantes :
  - des mesures ponctuelles du champ 2D non bruitées
  - un prédicteur spatialisé, qui consiste en un champ 2D bruité.
  - des mesures par tomographie obtenues le long de segments

Le but est d'adapter et de comparer deux méthodes d'apprentissage différentes basées sur des réseaux de neurones profonds. Pour simplifier, nous allons travailler sur des données de synthèse générées à la volée.

Ces données peuvent être visualisées grâce à la fonction gen_image_with_pairs.

In [14]:
from exam_S3.utile_Transformers import voir_batch2D, gen_image_with_pairs, set_tensor_values

batch_size = 6
n_points = 16
n_pairs = 16
full_target, point_measurements, spatial_predictor, line_measurements_viz, line_measurements = gen_image_with_pairs(6, n_lines, n_points)
# NB : Le code de gen_image_with_pairs est précompilé avec numba. Le premier run est donc nettement plus long que les suivants.

In [ ]:
# exemples de champ 2D cible complets (full_target)
# ils contiennent des disques, qu'il va s'agir de reconstruire au mieux
fig1 = plt.figure(1, figsize=(36, 6))
voir_batch2D(full_target, 6, fig1, k=0, min_scale=0, max_scale=1)

In [ ]:
# Pour reconstruire, on s'appuira sur des triplets contenant les positions et les
# valeurs de mesures ponctuelles (point_measurements).
# Précisément, ces triplets (x, y, m) contiennent :
# - les coordonnées x, y des mesures ponctuelles dans le repère (O, A, B)
# où O correspond au coin en bas à gauche de full_target, A au coin en bas à droite
# et B au coin en haut à gauche.
# - m : valeur au pixel de coordonnées (x,y) de full_target

# Nous avons généré batch_size x n_points triplets :
print(point_measurements.shape)

# Pour visualiser ces mesures, on peut utiliser la fonction set_tensor_values(t,point_measurements, size):
# qui affectent aux pixels de t de coordonnées x,y les valeurs m. Par exemple:
point_measurements_viz = set_tensor_values(torch.zeros((6,1,64,64)), point_measurements, 64)
fig2 = plt.figure(2, figsize=(36, 6))
voir_batch2D(point_measurements_viz , 6, fig2, k=0, min_scale=0., max_scale=0.5)
# NB: - bien noter le format utilisé pour le tenseur t
#     - il y a bien 16 points par images, mais la plupart correspondent à des
#       mesures nulles

In [ ]:
# On s'appuiera aussi sur des prédicteurs spatialisés bruités.
# Les rectangles figurent le bruit. Les disques contenus dans ces images
# sont alignés avec ceux du champ 2D à reconstruire
# mais leurs intensités sont différentes.

fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(spatial_predictor, 6, fig3, k=0, min_scale=0, max_scale=1)


In [ ]:
# Enfin, on s'appuie sur des mesures intégrées le long des segements contenus
# dans des quintuplets
# Précisément, ces quintuplets (x0, y0, x1, y1, Is) contiennent :
# - les coordonnées x0, y0 de la première extrémité du segment
# - les coordonnées x1, y1 de la seconde extrémité du segement
# la valeur moyenne I du champ 2D full_target le long du segment


# Nous avons ainsi généré batch_size x n_pairs quintuplets :
print(line_measurements.shape)


# Le tenseur line_measurements_viz permet de visualiser ces segments :
fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(line_measurements_viz, 6, fig3, k=0, min_scale=0, max_scale=1)

# NB: pour cette visualisation, les intensités des pixels par lesquels passent
# les segements ont été réglées sur 0.2 + Is  (sauf aux intersections)

## B. Attendus

gen_image_with_pairs permet d'aborder plusieurs problème d'apprentissage par plusieurs méthodes différentes.
Pb n°1

## Annexe : exemple d'un visual transformer adapté au problème

In [ ]:
# Paramètres du modèle :
image_size = [64,64]
channels = 1
patch_size = 4
d_model = 120
mlp_expansion_ratio = 4
d_ff = mlp_expansion_ratio * d_model
n_heads = 4
n_layers = 12

In [ ]:
# Module interne du réseau responsable de l'encodage des variables :
from PREAC.utile_Transformers import UnifiedEmbedding
ue = UnifiedEmbedding(d_model, patch_size, channels)
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)
embeddings = ue(radar, pluviometres, cmls)
print(embeddings.shape)


In [ ]:
from exam_S3.utile_Transformers import FusionTransformer
model = FusionTransformer(image_size, patch_size, n_layers, d_model, d_ff, n_heads, channels=1)
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)
model(radar, pluviometres, cmls).shape


def criterion(output, target):
    return torch.abs((output - target)).mean()

import torch.optim as optim
optimizer = optim.Adam(model.parameters(), 10**(-4))

## E. Chargement d'un Transformer entraîné

In [ ]:
# Avec France Transfert ??
# !curl 'https://francetransfert.numerique.gouv.fr/api-private/download-module/generate-download-url' -X POST \
# -H 'Content-Type: application/json' \
# -H 'Origin:https://francetransfert.numerique.gouv.fr' \
# --data-raw '{"enclosure":"164ea132-cf5e-4a8d-a084-62841b3122ec","recipient":"cGllcnJlLmxlcGV0aXRAbWV0ZW8uZnI%3D","token":"ddf68980-7b19-4eef-8a34-88a3e32a0f71","senderToken":null,"password":"2q*vbl62!FK@Z"}'

In [ ]:
# Modèles entraînés sur 900 époques :
# mViT_900ep.pth comme au D.
# mViT_0radar_900ep.pth avec, au préalable: radar = 0 x radar
! wget https://www.grosfichiers.com/K3aaxZcSnX4_Fic8rPjJ9yZ
! unzip K3aaxZcSnX4_Fic8rPjJ9yZ
! rm K3aaxZcSnX4_Fic8rPjJ9yZ

Bibliographie :  [Jaegle et al. 2020](https://arxiv.org/abs/1811.12739)